# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [16]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 34

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  34  
1 :  generation selection from the cross Sutter/Numar//Numar*4/Mildew Resistant White California Mariout.
2 :  Its  experimental designation was 76264.
3 :  The cross was designed to develop a Numar-type cultivar with resistance or tolerance to scald, net blotch, powdery mildew and BYD, together with improved straw quality.
4 :  UC 337 is mid-tall (2-3 inches taller then UC 476 and Prato) with moderately strong straw and medium early maturity (comparable to Prato and Briggs).
5 :  The basal rachis internode is straight.
6 :  Rachis edges are covered with hair.
7 :  The spike is parallel, dense and inclined at maturity.
8 :  The lemma awn is long and smooth.
9 :  The glume awn is one-half the length of the lemma.
10 :  The few rachilla hairs are long and occur mainly at the tip.
11 :  Kernels are mid-long to long with slightly wrinkled hulls and blue aleurone.
12 :  At the time of release UC 337 was resistant to moderately resistant to BYD, scald, net blotch, powdery mildew, a

## Read in Per-line named entity file and match entities to sentence positions.

In [18]:
import re
import csv
import pandas as pd
fname = "Data/barley_p34_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['1', 'Sutter/Numar//Numar*4/Mildew Resistant White California Mariout', 'PED']
Handle manually:  ['15', 'Crop Science 30:1154 (1990)', 'JRNL']
Handle manually:  ['36', 'Crop Science 30:1154 (1990)', 'JRNL']
Handle manually:  ['39', 'CM 67/Numar*2', 'PED']
Handle manually:  ['56', 'Crop Science 13:770 (1973)', 'JRNL']
Handle manually:  ['60', '[(Numar*2/CI 2376)/Yellow Dwarf Resistant Numar]/2*UC 75012W', 'PED']
Handle manually:  ['78', 'Crop Science 36:466 (1996)', 'JRNL']
    Index                                           Sentence  \
0       2           Its  experimental designation was 76264.   
1       3  The cross was designed to develop a Numar-type...   
2       3  The cross was designed to develop a Numar-type...   
3       3  The cross was designed to develop a Numar-type...   
4       3  The cross was designed to develop a Numar-type...   
..    ...                                                ...   
182    75  At the time of release it was moderately res

## Create a function to clean up overlapping intervals

In [19]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [20]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('The few rachilla hairs are long and occur mainly at the tip.', {'entities': [(8, 22, 'PLAN')]}),
    ('Kernels are mid-long to long with slightly wrinkled hulls and blue aleurone.', {'entities': [(0, 7, 'PLAN'), (52, 57, 'PLAN'), (67, 75, 'PLAN')]}),
    ('At the time of release UC 337 was resistant to moderately resistant to BYD, scald, net blotch, powdery mildew, and leaf rust.', {'entities': [(23, 29, 'CVAR'), (34, 43, 'PPTD'), (47, 67, 'PPTD'), (71, 74, 'PATH'), (76, 81, 'PATH'), (83, 93, 'PATH'), (95, 109, 'PATH'), (115, 124, 'PATH')]}),
    ('It subsequently became moderately susceptible to scald, leaf rust, and powdery mildew, and susceptible to stripe rust.', {'entities': [(23, 45, 'PPTD'), (49, 54, 'PATH'), (56, 65, 'PATH'), (71, 85, 'PATH'), (91, 102, 'PPTD'), (106, 117, 'PATH')]}),
    ('UC 476 is a six-rowed spring feed barley.', {'entities': [(0, 6, 'CVAR'), (12, 21, 'TRAT'), (22, 28, 'TRAT'), (29, 33, 'TRAT'), (34, 40, 'CROP')]}),
    ('It was released by the Univer